# DNC DATA EXPERT

# Dinâmica 01 - New York Flights

## CARREGAMENTO DAS BIBLIOTECAS

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib widget

## CARREGAMENTO DOS DADOS

In [2]:
df = pd.read_csv('D:\\Samuel\\Documentos\\Python\\dncPython\\dinamica\\Datasets\\D01\\nyflights.csv')

In [3]:
# Remover a primeira coluna pois o Python já possui indexação
df = df.drop('Unnamed: 0', 1)

In [4]:
df.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01 05:00:00
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01 05:00:00
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01 05:00:00
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01 05:00:00
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01 06:00:00


## QUESTÕES A SEREM RESPONDIDAS

01 - Quais são as métricas de centralização, variabilidade e quantis dos vôos que atrasam mais que duas horas para decolar? Responda de modo numérico e gráfico.

02 - Quais destinos geraram vôos com o maior número de atrasos na partida? Responda de modo qantitativo e gráfico. (5 piores | Considerar 5 minutos um atraso aceitável)

03 - Para os 5 piores destinos com maior número de voôes atrasados na partida, calcule as estatísticas descritivas da velocidade média durante o vôo.

04 - Aumentar a velocidade média dos voôs pode ser uma solução para diminuir os atrasos? Responda graficamente.

## QUESTÃO 01

### Filtrar a tabela: atraso > 2hrs

In [5]:
dfDelay = df[df['dep_delay'] > 120]

### Estatísticas descritivas para os atrados

In [6]:
delayDesc = dfDelay['dep_delay'].describe()

In [7]:
# Trasnformar as estatísticas descritivas em data frame
delayDesc = pd.DataFrame(delayDesc)

In [8]:
# Transpor o dataframe e adicionar nas meétricas faltantes
delayDesc = pd.DataFrame.transpose(delayDesc)

In [9]:
# Adicionar uma coluna para a amplitude
delayDesc['range'] = delayDesc['max'] - delayDesc['min']

In [10]:
# Adicionar uma coluna para variância
delayDesc['var'] = pow(delayDesc['std'], 2)

In [11]:
delayDesc

,count,mean,std,min,25%,50%,75%,max,range,var
dep_delay,9723.0,186.983236,74.596836,121.0,139.0,166.0,210.0,1301.0,1180.0,5564.687952


A média de atrasos para os vôos que atrasam mais que duas horas é de 187 minutos com desvio padrão de 75 minutos. A mediana é de 166 minutos. o maior atraso foi de 1301 minutos (21,86 horas... ouch!).

### Análise gráfica dos atrasos maiores que duas horas.

#### Histograma

In [12]:
# Cria a figura 'vazia'
fig = plt.figure()

# Plota o histograma
plt.hist(dfDelay['dep_delay'], bins = 20)

# Adiciona título e subtítulo
plt.suptitle('Duração dos atrasos na decolagem', fontsize = 16, fontweight = 'bold', x = 0.465)
plt.title('Valores maiores que duas horas', loc = 'left')

# Adiciona título aos eixos 
plt.xlabel('Minutos')
plt.ylabel('Frequência')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Frequência')

#### Box Plot

In [13]:
# Plot figura em branco
fig = plt.figure()

# Plota o gráfico de box plot

plt.boxplot(dfDelay['dep_delay'])

plt.ylabel('Minutos')

# Remove o valor do eixo x
plt.xticks([1], [''])

# Remove o tick do eixo x
plt.tick_params(axis = "x", which = "both", bottom = False, top = False)

plt.suptitle('Duração do atraso na decolagem', fontsize = 16, fontweight = 'bold', x = 0.445)
plt.title('Valores maiores que duas horas', loc = 'left')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.0, 1.0, 'Valores maiores que duas horas')

## QUESTÃO 02

### Filtrar tabela com vôos que atrasaram

In [14]:
dfDest = df[df['dep_delay'] > 5]

### Contar a quantidade de atrasos nos vôos

In [15]:
delayCounts = dfDest['dest'].value_counts()

In [16]:
delayCounts = pd.DataFrame(delayCounts)
delayCounts

,dest
ORD,4975
ATL,4687
LAX,4442
MCO,4083
SFO,4078
...,...
EYW,6
SBN,6
MTJ,6
ANC,3


In [17]:
# Transformar o índice das linhas em uma coluna
delayCounts['destino'] = delayCounts.index

# Mudar o nome da coluna de contagem
delayCounts.columns = ['atrasos', 'destino']

# Remover os valores do index
delayCounts.reset_index(drop = True, inplace = True)

# Mudar a sequência das colunas
delayCounts = delayCounts[['destino', 'atrasos']]

delayCounts

,destino,atrasos
0,ORD,4975
1,ATL,4687
2,LAX,4442
3,MCO,4083
4,SFO,4078
...,...,...
98,EYW,6
99,SBN,6
100,MTJ,6
101,ANC,3


Os 5 destinos que geraram mais vôos atrasados foram: Chicago (ORD), Atlanta (ATL), Los Angeles (LAX), Orlando (MCO) e São Francisco (SFO).

### Gráfico de barras 

In [18]:
fig = plt.figure()

plt.bar(delayCounts['destino'], delayCounts['atrasos'])

plt.ylabel('Atrasos')

# Remove o valor do eixo x
plt.xticks([1], [''])

# Remove o tick do eixo x
plt.tick_params(axis = "x", which = "both", bottom = False, top = False)

plt.suptitle('Atraso na decolagem', fontsize = 18, fontweight = 'bold', x = 0.36)
plt.title('Valores agrupados por diferentes aeroportos', loc = 'left')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.0, 1.0, 'Valores agrupados por diferentes aeroportos')

## QUESTÃO 03

### Filtrar a tabela somente com os piores destinos

In [19]:
# Seleciona os piores destinos
worstDest = delayCounts.sort_values(by = 'atrasos', ascending = False)[['destino']][0:4]

In [20]:
# Filtra a tabela original baseados nos destinos
worstDelay = df[df['dest'].isin(worstDest['destino'])]
worstDelay.reset_index(drop = True, inplace = True)

### Calculara velocidade média de cada vôo

In [21]:
worstDelay = worstDelay.assign(avgVel = (worstDelay['distance'] / worstDelay['air_time'])*60)

In [22]:
worstDelay

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour,avgVel
0,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01 06:00:00,394.137931
1,2013,1,1,554.0,558,-4.0,740.0,728,12.0,UA,1696,N39463,EWR,ORD,150.0,719,5,58,2013-01-01 05:00:00,287.600000
2,2013,1,1,557.0,600,-3.0,838.0,846,-8.0,B6,79,N593JB,JFK,MCO,140.0,944,6,0,2013-01-01 06:00:00,404.571429
3,2013,1,1,558.0,600,-2.0,753.0,745,8.0,AA,301,N3ALAA,LGA,ORD,138.0,733,6,0,2013-01-01 06:00:00,318.695652
4,2013,1,1,558.0,600,-2.0,924.0,917,7.0,UA,194,N29129,JFK,LAX,345.0,2475,6,0,2013-01-01 06:00:00,430.434783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64749,2013,9,30,2052.0,2059,-7.0,2315.0,2358,-43.0,UA,1416,N68453,EWR,MCO,123.0,937,20,59,2013-09-30 20:00:00,457.073171
64750,2013,9,30,2053.0,1815,158.0,2310.0,2054,136.0,EV,5292,N600QX,EWR,ATL,91.0,746,18,15,2013-09-30 18:00:00,491.868132
64751,2013,9,30,2121.0,2100,21.0,2349.0,14,-25.0,DL,2363,N193DN,JFK,LAX,296.0,2475,21,0,2013-09-30 21:00:00,501.689189
64752,2013,9,30,2140.0,2140,0.0,10.0,40,-30.0,AA,185,N335AA,JFK,LAX,298.0,2475,21,40,2013-09-30 21:00:00,498.322148


### Agrupar a tabela com os piores destinos

In [23]:
grouped = worstDelay.groupby(worstDelay['dest'])

### Criar estatísticas descritivas para velocidade média

In [24]:
grouped.describe()['avgVel']

,count,mean,std,min,25%,50%,75%,max
dest,,,,,,,,
ATL,16837.0,405.182104,33.994256,254.318182,381.000000,406.909091,430.384615,703.384615
LAX,16026.0,452.799331,25.048269,337.500000,435.483871,451.656442,469.936709,540.000000
MCO,13967.0,422.385530,32.109714,265.116279,401.571429,425.864662,445.984252,529.345794
ORD,16566.0,381.130263,31.117499,217.878788,360.491803,382.434783,403.486239,495.862069


### Plotar box-plot para os quatro destinos.

In [25]:
fig = plt.figure()

sns.boxplot(x = worstDelay['dest'], y = worstDelay['avgVel'])

plt.ylabel('')

plt.xlabel('Destino')

plt.suptitle('Velocidade dos piores destinos', fontsize = 18, fontweight = 'bold', x = 0.465)
plt.title('Milhas por hora', loc = 'left')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.0, 1.0, 'Milhas por hora')

## QUESTÃO 04 

### Criar tabela com os 'melhores destinos'

In [26]:
# Seleciona os melhores destinos
bestDest = delayCounts.sort_values(by = 'atrasos', ascending = True)[['destino']][0:5]

In [27]:
# Filtra a tabela original baseados nos destinos
bestDelay = df[df['dest'].isin(bestDest['destino'])]
bestDelay.reset_index(drop = True, inplace = True)

In [28]:
# Criar a coluna com a volocidade média
bestDelay = bestDelay.assign(avgVel = (bestDelay['distance'] / bestDelay['air_time'])*60)

In [29]:
# Criar uma coluna com a inscrição 'best'
bestDelay = bestDelay.assign(desType = 'best')

### Empilhar planilha com destinos bons e ruins.

In [30]:
# Adicionar coluna com a inscrição 'worst' nos priores destinos
worstDelay = worstDelay.assign(desType = 'worst')

In [31]:
# Crioar a mesma coluna com inscrição 'worst' nos destinos 'ruins'
allDelay = worstDelay.assign(desType = 'worst')

In [32]:
allDelay = worstDelay.append(bestDelay.reset_index(drop = True), ignore_index = True)

### Criar gráfico de box plot para explorar as diferenças

In [33]:
fig = plt.figure()

sns.boxplot(x = allDelay['desType'], y = allDelay['avgVel'])

plt.ylabel('Velocidade média')

plt.xlabel('Tipo de destino')

plt.suptitle('Comparação entre destinos', fontsize = 18, fontweight = 'bold', x = 0.425)
plt.title('Velocidade média durate o vôo', loc = 'left')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.0, 1.0, 'Velocidade média durate o vôo')

As velocidades médias do voôs que mais atrasam e que menos atrasam tende a ser a mesma. Por isso aumentá-la não parece ser uma boa solução inicialmente.